<a href="https://colab.research.google.com/github/Abdullah12389/Multimart/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
chat=pd.read_excel("data_chatbot.xlsx",usecols=['Question','Response'])

In [ ]:
chat.dropna(inplace=True)

In [ ]:
chat.isnull().sum()

,0
Question,0
Response,0


In [ ]:
chat.head()

,Question,Response
0,Say Jim how about going for a few beers after ...,You know that is tempting but is really not go...
1,What do you mean It will help us to relax,Do you really think so I don't It will just ma...
2,I guess you are right.But what shall we do I d...,I suggest a walk over to the gym where we can ...
3,That's a good idea I hear Mary and Sally often...,Sounds great to me If they are willing we coul...
4,Good.Let s go now,All right


In [ ]:
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
nltk.download('punkt_tab')
nltk.download('stopwords')
nlp=spacy.load('en_core_web_sm')
stopwords=stopwords.words("english")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def preprocess(text):
    text=re.sub(r"[^\w\s]","",text)
    text=text.lower()
    words=word_tokenize(text)
    return words

In [ ]:
chat['Question']=chat['Question'].astype(str).apply(preprocess)
chat['Response']=chat['Response'].astype(str).apply(preprocess)

In [ ]:
corpus=chat['Question'].tolist()
corpus+=chat['Response'].tolist()

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer()
tokenizer.fit_on_texts(corpus)

In [ ]:
len(tokenizer.word_index)

21443

In [ ]:
input_sequences=tokenizer.texts_to_sequences(chat['Question'])
output_sequences=tokenizer.texts_to_sequences(chat['Response'])

In [ ]:
input_sequences=pad_sequences(input_sequences,maxlen=20)
output_sequences=pad_sequences(output_sequences,maxlen=20)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM,Dense,Input,Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Attention
import numpy as np

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self):
    super(Encoder,self).__init__()
    self.h1=LSTM(5,return_sequences=True,return_state=True);
    self.h2=LSTM(5,return_sequences=True,return_state=True);
  def call(self,inputs):
    x,h_state1,c_state1=self.h1(inputs)
    x,h_state2,c_state2=self.h2(x)
    return x,[h_state1,h_state2],[c_state1,c_state2]

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self):
    super(Decoder,self).__init__()
    self.h1=LSTM(5,return_sequences=True,return_state=True);
    self.h2=LSTM(5,return_sequences=True,return_state=True);
    self.dense=Dense(5,activation="softmax")
  def call(self,inputs,h_state,c_state,encoder_outputs):
    context=Attention()([inputs,encoder_outputs])
    input=tf.concat([inputs,context],axis=-1)
    x,h_state1,c_state1=self.h1(input,initial_state=[h_state[0],c_state[0]])
    x,h_state2,c_state2=self.h2(x,initial_state=[h_state[1],c_state[1]])
    x=self.dense(x)
    return x,[h_state1, h_state2], [c_state1, c_state2]

In [ ]:
emb=Embedding(21444,5)
input=emb(input_sequences)
output=emb(output_sequences)

In [ ]:
encoder=Encoder()
decoder=Decoder()
inp=Input(shape=(20,5))
out=Input(shape=(20,5))
encoder_output,h_state,c_state=encoder(inp)
decoder_output,h_stated,c_stated=decoder(out,h_state,c_state,encoder_output)
model=Model([inp,out],decoder_output)

In [ ]:
model.compile(optimizer="adam",loss='mse',metrics=['accuracy'])
model.fit([input,output],output,epochs=10)

Epoch 1/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 33s 20ms/step - accuracy: 0.6144 - loss: 0.0389
Epoch 2/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - accuracy: 0.8333 - loss: 0.0386
Epoch 3/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 27s 20ms/step - accuracy: 0.8657 - loss: 0.0386
Epoch 4/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 27s 20ms/step - accuracy: 0.9860 - loss: 0.0385
Epoch 5/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - accuracy: 0.9900 - loss: 0.0385
Epoch 6/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - accuracy: 0.9916 - loss: 0.0385
Epoch 7/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 46s 24ms/step - accuracy: 0.9918 - loss: 0.0385
Epoch 8/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 30s 22ms/step - accuracy: 0.9923 - loss: 0.0386
Epoch 9/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 39s 20ms/step - accuracy: 0.9926 - loss: 0.0386
Epoch 10/10
1362/1362 ━━━━━━━━━━━━━━━━━━━━ 41s 20ms/step - accuracy: 0.9930 - loss: 0.0386


In [ ]:
encoder_input = input[0:1]  # or the appropriate slice of input
decoder_input = np.zeros_like(encoder_input)  # or any appropriate decoder input
pred=model.predict([encoder_input, decoder_input])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


In [ ]:
from gensim.models import Word2Vec
word2vec=Word2Vec(corpus,min_count=1,vector_size=5)

In [ ]:
for i in range(20):


ValueError: operands could not be broadcast together with shapes (100,) (5,) (100,) 

In [ ]:
pred

array([[[0.2004994 , 0.19998084, 0.20095973, 0.2001263 , 0.1984336 ],
        [0.20075731, 0.20006703, 0.20109409, 0.19954275, 0.19853868],
        [0.2005639 , 0.1996525 , 0.20098853, 0.19988257, 0.19891253],
        [0.20055746, 0.199772  , 0.20097059, 0.1998632 , 0.19883671],
        [0.20057866, 0.1997805 , 0.20097432, 0.19985865, 0.19880784],
        [0.20059371, 0.1997813 , 0.20098005, 0.19985765, 0.1987874 ],
        [0.20060523, 0.1997842 , 0.20098458, 0.19985723, 0.19876887],
        [0.20061374, 0.19978712, 0.20098853, 0.19985715, 0.19875342],
        [0.20062031, 0.19979002, 0.20099191, 0.19985728, 0.1987405 ],
        [0.20062542, 0.19979255, 0.20099472, 0.19985746, 0.19872986],
        [0.2006295 , 0.19979477, 0.20099705, 0.19985756, 0.19872099],
        [0.20063286, 0.19979662, 0.200999  , 0.1998578 , 0.1987138 ],
        [0.20063545, 0.1997982 , 0.20100053, 0.19985794, 0.1987078 ],
        [0.20063773, 0.1997995 , 0.2010019 , 0.19985802, 0.19870295],
        [0.20063944,